# Import

In [90]:
import pandas as pd

In [91]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [92]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")  
submission = pd.read_csv("./data/sample_submission.csv")  

## 영어 번역 내용 제거

기사, 구글 순서일때

In [93]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

영문, 번역, 오류, 전제 순서일때

In [94]:
# '키워드' 열에서 '영문', '번역', '오류', '전제' 네 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['영문', '번역', '오류', '전제']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['영문', '번역', '오류', '전제']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '영문, 번역, 오류, 전제' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['영문', '번역', '오류', '전제']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('영문')].strip()

# 각 행의 '키워드' 값에서 '영문' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

구글, 번역기, 번역, 영문기사 순서일때

In [95]:
# '키워드' 열에서 '구글', '번역기', '번역', '영문기사' 네 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['구글', '번역기', '번역', '영문기사']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['구글', '번역기', '번역', '영문기사']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '구글, 번역기, 번역, 영문기사' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['구글', '번역기', '번역', '영문기사']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('구글')].strip()

# 각 행의 '키워드' 값에서 '구글' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

구글번역, 번역 순서일때

In [96]:
# '키워드' 열에서 '구글번역', '번역' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['구글번역', '번역']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['구글번역', '번역']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '구글번역, 번역' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['구글번역', '번역']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('구글번역')].strip()

# 각 행의 '키워드' 값에서 '구글번역' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

한글 -> 영어(5단어이상 이어질때) 처리

In [97]:
import pandas as pd
import re

def remove_english_after_korean(text):
    # 한글과 영어 단어를 구분하는 정규 표현식
    korean_pattern = re.compile(r'[가-힣]+')
    english_pattern = re.compile(r'[a-zA-Z]+')
    
    korean_count = 0
    english_count = 0
    start_index = 0
    
    words = text.split(',')
    
    for i, word in enumerate(words):
        if korean_pattern.search(word):
            korean_count += 1
        elif english_pattern.search(word):
            english_count += 1
            if english_count == 1:
                start_index = i
            if english_count >= 5:
                return ','.join(words[:start_index])
    
    return text

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_english_after_korean)
test_df['키워드'] = test_df['키워드'].apply(remove_english_after_korean)

기사,Google,번역,번역, 으로 남아있는 문장에 대한 처리

In [98]:
import pandas as pd

def remove_last_keywords(text):
    keywords_to_remove = ['기사', 'Google', '번역', '번역']
    words = text.strip(',').split(',')
    
    # 마지막 4개의 단어가 keywords_to_remove와 일치하는지 확인
    if words[-4:] == keywords_to_remove:
        return ','.join(words[:-4]) + ',' if len(words[:-4]) > 0 else ''
    
    return text

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_last_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_last_keywords)

데이터 저장

In [99]:
# 수정된 데이터프레임을 CSV 파일로 저장
train_df.to_csv('./data/train_df_translate_del.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('./data/test_df_translate_del.csv', index=False, encoding='utf-8-sig')

.